# Chatbot QA con LangChain (y LangSmith)

![Clases de Langchain vinculadas a la etapa de ingesta de contenidos](https://github.com/lia-ve/agentes-langchain/blob/main/img/langchain_ingestion.jpg?raw=true)

Aquí están los componentes mostrados en la figura:

- **Document**: Modela el contenido del texto y los metadatos relacionados.
- **BaseLoader**: Carga texto desde fuentes externas en el modelo de documento.
- **TextSplitter (Divisor de Texto)**: Divide los documentos en fragmentos más pequeños para un procesamiento eficiente.
- **VectorStore**: Almacena fragmentos de texto y sus embeddings relacionados para una recuperación eficiente.
- **Embeddings**: Convierte el texto en embeddings (representaciones vectoriales).

![Clases de Langchain vinculadas a la etapa de recuperación y generación](https://github.com/lia-ve/agentes-langchain/blob/main/img/langchain_gen.jpg?raw=true)

La figura incluye los siguientes componentes:

- **Vector Store**: Almacena y recupera fragmentos de texto relevantes.
- **Retriever**: Recupera fragmentos de texto relevantes basándose en la similitud entre el embedding de la consulta y los embeddings de texto almacenados.
- **Embedding Model**: Asegura embeddings consistentes para las consultas y documentos.
- **Prompt / PromptTemplate**: Construye la entrada para el modelo de lenguaje, utilizando típicamente la pregunta del usuario y un contexto formado por los fragmentos de texto recuperados.
- **Chat Model / LLM**: Genera respuestas utilizando el contexto y la consulta proporcionados.

In [ ]:
# %pip install langchain langchain_community langchain-chroma langchain_openai langchain-unstructured chromadb docx2txt pypdf wikipedia unstructured

In [ ]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

##  Ingesta de Contenidos

In [ ]:
import os

from langchain_community.document_loaders import WikipediaLoader, Docx2txtLoader, PyPDFLoader, TextLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("LIA_API_KEY"),openai_api_base=os.getenv("LIA_EMBEDDING_API_BASE"), model=os.getenv("EMBEDDING_MODEL"))
vector_db = Chroma("info_anu", embeddings_model)


### Carga de Documentos

Procesa cada documento cargándolo, dividiéndolo en fragmentos, convirtiendo los fragmentos en vectores y guardando en la base de datos vectorial.

In [ ]:
# Hagamos una utility para dividir y cargar documentos

def split_and_import(loader):
     chunks = text_splitter.split_documents(loader.load())
     vector_db.add_documents(chunks)
     print(f"Fragmentos cargados de la fuente {loader}")

In [ ]:
# Carga de documentos de Wikipedia

wikipedia_loader = WikipediaLoader(query="Pueblo Añú")
split_and_import(wikipedia_loader)

In [ ]:
# Carga de documentos de otros formatos

word_loader = Docx2txtLoader("../../datasets/anu/pueblo_anu.docx")
split_and_import(word_loader)
 
pdf_loader = PyPDFLoader("../../datasets/anu/pautas-crianza-pueblo-anu-venezuela-completo.pdf")
split_and_import(pdf_loader)
 
pdf_loader = PyPDFLoader("../../datasets/anu/tesis-anu-reducido.pdf")
split_and_import(pdf_loader)
 
txt_loader = TextLoader("../../datasets/anu/pueblo_anu.txt")
split_and_import(txt_loader)

### Cargar documentos desde un directorio

In [ ]:
# Mapeo de extensiones de archivo a clases de cargadores de documentos
loader_classes = {
    'docx': Docx2txtLoader,
    'pdf': PyPDFLoader,
    'txt': TextLoader
}

# Clase para cargar todos los documentos en un directorio
class DirectoryLoader:
    def __init__(self, folder_path, vector_db):
        self.folder_path = folder_path
        self.vector_db = vector_db
    
    def get_loader(self, filename):
        """Devuelve el cargador apropiado para el archivo basado en su extensión."""
        _, file_extension = os.path.splitext(filename)
        file_extension = file_extension.lstrip('.')
        
        loader_class = loader_classes.get(file_extension)
        
        if loader_class:
            return loader_class(filename)
        else:
            raise ValueError(f"No hay cargador disponible para la extensión de archivo '{file_extension}'")

    def load_all_documents(self):
        """Carga todos los documentos compatibles en el directorio y los agrega a la base de datos vectorial."""
        for filename in os.listdir(self.folder_path):
            file_path = os.path.join(self.folder_path, filename)
            if os.path.isfile(file_path):  # Asegurarse de que es un archivo
                try:
                    loader = self.get_loader(file_path)  # Obtener el cargador apropiado
                    chunks = text_splitter.split_documents(loader.load())
                    self.vector_db.add_documents(chunks)
                    print(f"Fragmentos cargados de la fuente {filename}")
                except ValueError as e:
                    print(e)  # Manejar extensiones de archivo no compatibles

In [ ]:
# Uso de DirectoryLoader para cargar todos los documentos dentro de una carpeta

directory_loader = DirectoryLoader("../../datasets/anu", vector_db)
directory_loader.load_all_documents()

In [ ]:
# Realicemos una prueba de búsqueda en la base de datos vectorial

query = "Cómo son las viviendas Añú?"
results = vector_db.similarity_search(query, 4)
print(results)

### Realizar una pregunta con una Cadena

![Cadena de RAG](https://github.com/lia-ve/agentes-langchain/blob/main/img/rag_chain.png?raw=true)

Componentes de la cadena RAG:

- **Retriever**: Recupera contenido de texto relevante de la base de datos vectorial y lo inyecta en el parámetro "contexto" del prompt.
- **Alimentador de Preguntas**: Implementado como un componente de "paso directo", pasa la pregunta del usuario a través de la interfaz `Runnable` (una clase abstracta en la que se basa cada componente de LangChain).
- **Modelo de Chat**: Procesa el prompt para generar la respuesta.

In [ ]:
# Primero, definamos la plantilla
from langchain_core.prompts import PromptTemplate

rag_prompt_template = """Utiliza las siguientes piezas de contexto para responder la pregunta al final.
Si no sabes la respuesta, simplemente di que no sabes; no intentes inventar una respuesta.
Usa un máximo de tres oraciones y mantén la respuesta lo más concisa posible.
{context}
Pregunta: {question}
Respuesta útil:"""

rag_prompt = PromptTemplate.from_template(rag_prompt_template)

In [ ]:
# Alternativamente puedes una plantilla de LangChain Hub. El problema es que hay pocas en español.
# from langchain import hub
# rag_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
retriever = vector_db.as_retriever()
 
from langchain_core.runnables import RunnablePassthrough
question_feeder = RunnablePassthrough()
 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
)

In [ ]:
rag_chain = {"context": retriever, "question": question_feeder} | rag_prompt | llm

In [ ]:
def execute_chain(chain, question):
    answer = chain.invoke(question)
    return answer.content

In [ ]:
# Probe la cadena con una pregunta

question = "Qué tipo de viviendas usan el pueblo añú?"
answer = execute_chain(rag_chain, question)
print(answer)

In [ ]:
# Nuestro chatbot no tiene memoria

question = "Repite tu respuesta anterior"
answer = execute_chain(rag_chain, question)
print(answer)

### Memoria

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente útil, experto en poblaciones indígenas venezolanas, especialmente en las comunidades ubicadas en la región occidental de Venezuela. Si no sabes la respuesta, simplemente di que no sabes; no intentes inventar una respuesta."),
        ("placeholder", "{chat_history_messages}"),
        ("assistant", "{retrieved_context}"),
        ("human", "{question}"),
    ]
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableLambda

chat_history_memory = ChatMessageHistory()

In [ ]:
# Actualicemos nuestra función de ejecución de la cadena para incluir la memoria

def execute_chain_with_memory(chain, question):
    chat_history_memory.add_user_message(question)
    answer = chain.invoke(question)
    chat_history_memory.add_ai_message(answer)
    return answer.content

In [ ]:
# Hacemos esta función porque langchain debe recibir un objeto RunnableLambda

def get_messages(x):
    return chat_history_memory.messages

In [ ]:
rag_chain_memory = {
    "retrieved_context": retriever, 
    "question": question_feeder,
    "chat_history_messages": RunnableLambda(get_messages)
} | rag_prompt | llm

In [ ]:
# Realicemos las preguntas de nuevo con la nueva cadena
question = "Qué tipo de viviendas usan el pueblo añú?"
answer = execute_chain_with_memory(rag_chain_memory, question)
print(answer)

In [ ]:
question = "Repite tu respuesta anterior"
answer = execute_chain(rag_chain_memory, question)
print(answer)

## Monitorear la solución con LangSmith

In [ ]:
# Definir estas variables de entorno en un archivo .env

# LANGSMITH_API_KEY=<API_KEY>
# LANGSMITH_TRACING=true
# LANGCHAIN_PROJECT=<NOMBRE_DEL_PROYECTO>
# LANGCHAIN_TRACING_V2=true

Con las variables en el archivo .env, puedes ejecutar todo el código y cada traza será enviada a LangSmith. Sin embargo si quieres tener más control de cada traza (por ejemplo ponerle un nombre a cada uno y mejorar el monitoreo), puedes hacerlo de la siguiente manera:

In [ ]:
from langsmith import trace
from langsmith import Client, traceable

In [ ]:
langsmith_client = Client(
    api_key=os.getenv("LANGSMITH_API_KEY"),
    api_url="https://api.smith.langchain.com",
)

In [ ]:
question = "Qué tipo de viviendas usan el pueblo añú?. Además, cita la fuente"
with trace("Chat Pipeline", "chain", project_name="Q&A chatbot 2", inputs={"input": question}, client=langsmith_client) as rt:
    answer = execute_chain(rag_chain, question)
    print(answer)
    rt.end(outputs={"output": answer})

### Creando un Chatbot QA con RetrievalQA

In [28]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
   llm=llm,
   chain_type="stuff",
   retriever=retriever,
   return_source_documents=False
)

In [34]:
# Vamos a definir de nuevo la función de ejecución de la cadena, para eliminar el retorno de answer.content, pues con RetrievalQA daría error

def execute_chain(chain, question):
    answer = chain.invoke(question)
    return answer

In [35]:
question = "Qué tipo de viviendas usan el pueblo añú?. Además, cita la fuente"
with trace("RetrievalQA", "chain", project_name="Q&A chatbot 2", inputs={"input": question}, client=langsmith_client) as rt:
    answer = execute_chain(rag_chain, question)
    print(answer)
    rt.end(outputs={"output": answer})

{'query': 'Qué tipo de viviendas usan el pueblo añú?. Además, cita la fuente', 'result': 'El pueblo Añú utiliza palafitos como tipo de vivienda. Según el texto, "los palafitos son modelos de viviendas, creadas por el ingenio de ésta etnia". La fuente es La Salle, 1983, p. 20.'}


In [36]:
# Puedes configurar el retriever de manera más avanzada, por ejemplo, para buscar solo documentos con una puntuación de similitud superior a 0.8 y devolver los 3 documentos más relevantes

retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8, 'k': 3}
)